In [14]:
import pandas as pd
import os
import random

In [15]:
import pandas as pd
import random
from datetime import datetime, timedelta

# LISTA DE ID DE CLIENTES
ids = list(range(1, 101))  # 100 clientes

# FUNÇÃO PARA GERAR DATA ALEATÓRIA
def gerar_data_aleatoria():
    inicio = datetime.strptime("2020-01-01", "%Y-%m-%d")
    fim = datetime.strptime("2023-12-31", "%Y-%m-%d")
    return (inicio + timedelta(days=random.randint(0, (fim - inicio).days))).strftime("%Y-%m-%d")

# GERA OS DADOS
id_cliente = [random.choice(ids) for _ in range(1000)]
valor_compra = [round(random.uniform(10.0, 1000.0), 2) for _ in range(1000)]
data_compra = [gerar_data_aleatoria() for _ in range(1000)]

# CRIA O DATAFRAME
dados = {
    "id_cliente": id_cliente,
    "valor_compra": valor_compra,
    "data_compra": data_compra
}

df = pd.DataFrame(dados)

# SALVA O CSV
df.to_csv("../csv/vendas.csv", index=False)

print("Arquivo 'vendas.csv' criado!")


Arquivo 'vendas.csv' criado!


In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, col, desc

# CRIAR A SESSÃO SPARK

spark = SparkSession.builder.appName("DesafioFinal").getOrCreate()

# LÊ O ARQUIVO CSV

df = spark.read.csv("../csv/vendas.csv", header=True, inferSchema=True)

# MOSTRA OS DADOS

df.show(10)

+----------+------------+-----------+
|id_cliente|valor_compra|data_compra|
+----------+------------+-----------+
|         4|      961.24| 2020-09-03|
|        42|      102.13| 2020-04-25|
|        19|       90.23| 2023-09-22|
|        33|      334.45| 2020-01-22|
|        98|      438.55| 2023-05-31|
|        62|       56.89| 2021-10-14|
|        90|      755.41| 2021-03-13|
|         3|      852.24| 2022-07-21|
|        58|      206.94| 2020-09-01|
|        65|      674.63| 2023-06-24|
+----------+------------+-----------+
only showing top 10 rows



In [17]:
# IDENTIFICA OS CLIENTES COM MAIOR VALOR DE COMPRA

clientes_maiores_compras = df.orderBy(desc("valor_compra"))
clientes_maiores_compras.show(10)

+----------+------------+-----------+
|id_cliente|valor_compra|data_compra|
+----------+------------+-----------+
|        41|      998.51| 2021-04-14|
|        59|      998.46| 2020-12-17|
|        56|       997.2| 2023-03-26|
|        78|      994.32| 2020-06-17|
|        80|      993.53| 2022-02-12|
|        78|      993.51| 2021-01-11|
|        86|      993.29| 2020-07-22|
|        82|      993.18| 2020-04-21|
|         8|      990.24| 2020-10-31|
|         7|      989.56| 2021-07-02|
+----------+------------+-----------+
only showing top 10 rows



In [18]:
# AGRUPA AS COMPRAS POR ANO E CALCULA O TOTAL

df_com_ano = df.withColumn("ano", year(col("data_compra")))
vendas_anuais = df_com_ano.groupBy("ano").sum("valor_compra")
vendas_anuais = vendas_anuais.withColumnRenamed("sum(valor_compra)", "total_vendas")
vendas_anuais.show()

+----+------------------+
| ano|      total_vendas|
+----+------------------+
|2023|133305.52000000002|
|2022|         117650.51|
|2020|126569.51999999999|
|2021|108649.27999999997|
+----+------------------+



In [19]:
# SALVA OS RESULTADOS

clientes_maiores_compras_pd = clientes_maiores_compras.toPandas()
vendas_anuais_pd = vendas_anuais.toPandas()

clientes_maiores_compras_pd.to_csv("../csv/clientes_maiores_compras.csv", index=False)
vendas_anuais_pd.to_csv("../csv/vendas_anuais.csv", index=False)

print("Arquivos salvos com sucesso!")


Arquivos salvos com sucesso!
